In [5]:
from sklearn import datasets


iris = datasets.load_iris()
X = iris.data
y = iris.target

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [7]:
import sys
sys.path.append("../") 

# Stochastic Gradient Descent

In [12]:
import mxnet as mx
import numpy as np
from mxnet import ndarray as nd
from hamiltonian.inference.sgd import sgd
from hamiltonian.models.softmax import softmax

model_ctx=mx.cpu()
hyper={'alpha':1}
in_units=[X.shape[1],1]
out_units=len(np.unique(y))


In [14]:
model=softmax(hyper,in_units,out_units,ctx=model_ctx)
inference=sgd(model,model.par,step_size=0.01,ctx=model_ctx)

In [15]:
par,loss=inference.fit(epochs=1000,batch_size=60,gamma=0.9,
                       X_train=nd.array(X_train,ctx=model_ctx),y_train=nd.array(y_train,ctx=model_ctx),verbose=False)

100%|██████████| 1000/1000 [00:07<00:00, 133.93it/s]


In [16]:
import hamiltonian
import importlib

try:
    importlib.reload(hamiltonian.models.softmax)
    importlib.reload(hamiltonian.inference.sgd)
    print('modules re-loaded')
except:
    print('no modules loaded yet')

modules re-loaded


In [17]:
model.net.collect_params()

{'1.weight': Parameter (shape=(3, 4), dtype=float32),
 '1.bias': Parameter (shape=(3,), dtype=float32)}

In [18]:
y_pred=model.predict(model.par,nd.array(X_test,ctx=model_ctx))

In [19]:
from sklearn.metrics import classification_report

print(classification_report(np.int32(y_test),np.int32(y_pred.sample().asnumpy())))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.83      0.91      0.87        11
           2       0.90      0.82      0.86        11

    accuracy                           0.90        30
   macro avg       0.91      0.91      0.91        30
weighted avg       0.90      0.90      0.90        30



# Hamiltonian Monte Carlo

In [29]:
model_ctx=mx.cpu()
hyper={'alpha':25.}
in_units=[X.shape[1],1]
out_units=len(np.unique(y))

In [30]:
from hamiltonian.inference.hmc import hmc
from hamiltonian.models.softmax import softmax

model=softmax(hyper,in_units,out_units,ctx=model_ctx)
inference=hmc(model,model.par,step_size=0.1,ctx=model_ctx)

In [31]:
import hamiltonian
import importlib

try:
    importlib.reload(hamiltonian.models.softmax)
    importlib.reload(hamiltonian.inference.hmc)
    print('modules re-loaded')
except:
    print('no modules loaded yet')

modules re-loaded


In [32]:
samples=inference.sample(epochs=3000,burn_in=1000,path_length=1.0,chains=4,X_train=nd.array(X_train,ctx=model_ctx),y_train=nd.array(y_train,ctx=model_ctx))

100%|██████████| 4000/4000 [01:48<00:00, 36.86it/s]


In [33]:
len(samples)

4

In [34]:
import arviz as az

datasets=[az.convert_to_inference_data(sample) for sample in samples]

In [35]:
dataset = az.concat(datasets, dim="chain")

In [36]:
az.summary(dataset)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"1.weight[0,0]",-7.056,20.094,-34.199,22.528,10.037,7.689,4.0,4.0,7.128774e+15
"1.weight[0,1]",19.531,16.355,2.257,44.474,8.169,6.258,4.0,4.0,5.573328e+15
"1.weight[0,2]",-53.708,23.179,-82.892,-18.169,11.578,8.870,4.0,4.0,7.128774e+15
"1.weight[0,3]",-23.018,9.620,-33.999,-7.621,4.805,3.681,4.0,4.0,5.573328e+15
"1.weight[1,0]",6.788,8.489,-6.878,15.597,4.240,3.248,4.0,4.0,5.573328e+15
"1.weight[1,1]",-8.613,10.146,-23.316,5.273,5.068,3.883,4.0,4.0,5.573328e+15
"1.weight[1,2]",19.112,3.539,14.182,22.795,1.768,1.354,4.0,4.0,7.128774e+15
"1.weight[1,3]",5.788,1.740,3.502,8.400,0.869,0.666,4.0,4.0,5.573328e+15
"1.weight[2,0]",0.454,17.906,-18.488,29.164,8.944,6.852,4.0,4.0,5.573328e+15
"1.weight[2,1]",-10.612,8.994,-22.732,0.186,4.492,3.442,4.0,4.0,5.573328e+15


# PyMC3

In [19]:
import pymc3 as pm
import theano.tensor as tt

In [22]:
with pm.Model() as model:
    alpha = pm.Normal('alpha', mu=0, sd=5, shape=3)
    beta = pm.Normal('beta', mu=0, sd=5, shape=(4,3))
    mu = pm.Deterministic('mu', alpha + pm.math.dot(X_train, beta))
    theta = tt.nnet.softmax(mu)
    y_hat = pm.Categorical('y_hat', p=theta, observed=y_train)
    trace_s = pm.sample(2000)

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, alpha]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 81 seconds.
There were 266 divergences after tuning. Increase `target_accept` or reparameterize.
There were 272 divergences after tuning. Increase `target_accept` or reparameterize.
There were 317 divergences after tuning. Increase `target_accept` or reparameterize.
There were 427 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


In [24]:
az.summary(trace_s,var_names=["alpha", "beta"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],1.109,4.913,-7.762,10.432,0.226,0.160,498.0,751.0,1.01
alpha[1],4.529,4.465,-2.828,13.372,0.328,0.232,208.0,585.0,1.02
alpha[2],-5.535,4.041,-13.071,2.113,0.110,0.078,1300.0,1394.0,1.02
"beta[0,0]",1.918,3.540,-4.650,8.760,0.140,0.099,644.0,1237.0,1.02
"beta[0,1]",0.876,3.121,-4.801,6.874,0.114,0.081,755.0,1176.0,1.01
"beta[0,2]",-2.448,3.198,-8.691,3.249,0.141,0.100,513.0,1551.0,1.01
"beta[1,0]",4.830,4.026,-1.753,12.752,0.225,0.159,316.0,1765.0,1.01
"beta[1,1]",-0.782,3.304,-7.066,5.150,0.174,0.123,381.0,1067.0,1.02
"beta[1,2]",-4.240,3.320,-10.575,2.185,0.112,0.079,885.0,1181.0,1.01
"beta[2,0]",-6.674,3.582,-13.903,-0.192,0.119,0.090,895.0,987.0,1.01
